# Model using Neural Network

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [2]:
df = pd.read_csv('cleaned_data.csv')

## Check the Columns' Data Type

In [3]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'fitness_goal'],
      dtype='object')

In [4]:
from sklearn.utils import shuffle

shuffled_df = shuffle(df, random_state = 42)

## Split Data for Training and Validation


In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(shuffled_df, test_size = 0.2, random_state=42)

## The training and validation set size

In [6]:
print('training : ',train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181


## Function Transform Dataframe into Tensor Dataset

In [7]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds

## Transform Training and Validation Dataframe into Tensor Dataset

In [8]:
train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [9]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [10]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [11]:
def encode_numerical_feature(column, name, dataset):

    normalizer = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = normalizer(column)

    return encoded_column

## Creating Tensor object from the column

In [12]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')

In [13]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight ]

## Encode columns based on their data type

In [14]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)


In [15]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded])

## Build the model architecture

In [22]:
x = keras.layers.Dense(32, activation = 'relu')(all_features)
x = keras.layers.Dense(16, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [23]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [ ]:
from IPython.core.display import display, HTML

# Enable horizontal scrolling for output
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Generate a long output
long_output = "A" * 1000

model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 3s 24ms/step - loss: 0.7376 - accuracy: 0.5367 - val_loss: 0.7091 - val_accuracy: 0.4917
Epoch 2/300
37/37 [==============================] - 0s 7ms/step - loss: 0.7217 - accuracy: 0.4938 - val_loss: 0.7044 - val_accuracy: 0.5083
Epoch 3/300
37/37 [==============================] - 0s 7ms/step - loss: 0.7184 - accuracy: 0.4606 - val_loss: 0.7012 - val_accuracy: 0.5028
Epoch 4/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6938 - accuracy: 0.5311 - val_loss: 0.6979 - val_accuracy: 0.5028
Epoch 5/300
37/37 [==============================] - 0s 7ms/step - loss: 0.7032 - accuracy: 0.5104 - val_loss: 0.6944 - val_accuracy: 0.5414
Epoch 6/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6895 - accuracy: 0.5214 - val_loss: 0.6939 - val_accuracy: 0.5359
Epoch 7/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6954 - accuracy: 0.5021 - val_loss: 0.6922 - val_accuracy: 0.5525
Epoch 8/300


Epoch 59/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6656 - accuracy: 0.5934 - val_loss: 0.6947 - val_accuracy: 0.5525
Epoch 60/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6633 - accuracy: 0.5795 - val_loss: 0.6964 - val_accuracy: 0.5359
Epoch 61/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6637 - accuracy: 0.5685 - val_loss: 0.6993 - val_accuracy: 0.5359
Epoch 62/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6683 - accuracy: 0.5906 - val_loss: 0.6958 - val_accuracy: 0.5304
Epoch 63/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6661 - accuracy: 0.5851 - val_loss: 0.6971 - val_accuracy: 0.5414
Epoch 64/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6650 - accuracy: 0.5920 - val_loss: 0.6950 - val_accuracy: 0.5359
Epoch 65/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6592 - accuracy: 0.5864 - val_loss: 0.6970 - val_accuracy: 0.5304
Epoch 

37/37 [==============================] - 0s 7ms/step - loss: 0.6573 - accuracy: 0.5920 - val_loss: 0.6999 - val_accuracy: 0.5193
Epoch 117/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6550 - accuracy: 0.6266 - val_loss: 0.7064 - val_accuracy: 0.5083
Epoch 118/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6576 - accuracy: 0.6252 - val_loss: 0.7015 - val_accuracy: 0.5028
Epoch 119/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6511 - accuracy: 0.6058 - val_loss: 0.7033 - val_accuracy: 0.4972
Epoch 120/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6586 - accuracy: 0.5768 - val_loss: 0.6976 - val_accuracy: 0.5028
Epoch 121/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6599 - accuracy: 0.5906 - val_loss: 0.6984 - val_accuracy: 0.5028
Epoch 122/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6593 - accuracy: 0.5892 - val_loss: 0.7005 - val_accuracy: 0.5083
Epoch 123/300

37/37 [==============================] - 0s 7ms/step - loss: 0.6372 - accuracy: 0.6100 - val_loss: 0.7108 - val_accuracy: 0.5249
Epoch 174/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6420 - accuracy: 0.6086 - val_loss: 0.7093 - val_accuracy: 0.5083
Epoch 175/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6465 - accuracy: 0.6100 - val_loss: 0.7091 - val_accuracy: 0.5083
Epoch 176/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6507 - accuracy: 0.5947 - val_loss: 0.7134 - val_accuracy: 0.5083
Epoch 177/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6469 - accuracy: 0.6169 - val_loss: 0.7133 - val_accuracy: 0.5304
Epoch 178/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6417 - accuracy: 0.6086 - val_loss: 0.7157 - val_accuracy: 0.5193
Epoch 179/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6430 - accuracy: 0.6058 - val_loss: 0.7148 - val_accuracy: 0.5083
Epoch 180/300

37/37 [==============================] - 0s 8ms/step - loss: 0.6284 - accuracy: 0.6515 - val_loss: 0.7212 - val_accuracy: 0.4972
Epoch 231/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6331 - accuracy: 0.6584 - val_loss: 0.7239 - val_accuracy: 0.5083
Epoch 232/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6378 - accuracy: 0.6155 - val_loss: 0.7276 - val_accuracy: 0.5249
Epoch 233/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6304 - accuracy: 0.6279 - val_loss: 0.7267 - val_accuracy: 0.5083
Epoch 234/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6294 - accuracy: 0.6183 - val_loss: 0.7292 - val_accuracy: 0.5083
Epoch 235/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6338 - accuracy: 0.6183 - val_loss: 0.7310 - val_accuracy: 0.5083
Epoch 236/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6368 - accuracy: 0.6141 - val_loss: 0.7219 - val_accuracy: 0.5359
Epoch 237/300

In [ ]:
df.info()

In [69]:
input_data = {
    'gender' : 1,
    'age'    :21,
    'workout_experience' : 0,
    'workout_time' : 60,
    'weight' : 65,
    'height' : 170,
}

In [70]:
input_dict = {}

In [71]:
for feature, value in input_data.items() :
    input_dict[feature] = tf.convert_to_tensor([value])

In [72]:
input_dict

{'gender': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>,
 'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([21])>,
 'workout_experience': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 'workout_time': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([60])>,
 'weight': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([65])>,
 'height': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([170])>}

In [76]:
predictions = model.predict(input_dict)

1/1 [==============================] - 0s 34ms/step


In [77]:
predictions

array([[0.5033811]], dtype=float32)

In [78]:
if predictions[0][0] >= 0.5 :
    print("You should do muscle up !")
else :
    print("You should do weight loss !")

You should do muscle up !


In [21]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'model_1_v2.pkl')

INFO:tensorflow:Assets written to: ram://a02c0d99-312e-4275-b03e-2019695547c0/assets


['model_1_v2.pkl']